In [1]:
# AI Models Python Notebook Setup Guide
# Installation & Imports
"""
#!pip install openai anthropic google-generativeai mistralai python-dotenv
"""

'\n#!pip install openai anthropic google-generativeai mistralai python-dotenv\n'

In [2]:
# Import libraries
import os
import json
import time  
from typing import Dict, Any, List
from dotenv import load_dotenv
import asyncio
from openai import OpenAI, AsyncOpenAI
import anthropic
import google.generativeai as genai
from mistralai import Mistral

load_dotenv()


True

In [3]:
# ==================== 1. OpenAI Setup ====================
class OpenAIClient:
    def __init__(self, model: str = "gpt-4o"):  # Updated to current model
        self.client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
        self.async_client = AsyncOpenAI(api_key=os.getenv('OPENAI_API_KEY'))
        self.model = model
        
    def chat_completion(self, messages: List[Dict], **kwargs) -> Dict[str, Any]:
        """Basic chat completion with error handling"""
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                **kwargs
            )
            return {
                "content": response.choices[0].message.content,
                "usage": {
                    "prompt_tokens": response.usage.prompt_tokens,
                    "completion_tokens": response.usage.completion_tokens,
                    "total_tokens": response.usage.total_tokens
                },
                "model": response.model
            }
        except Exception as e:
            return self._handle_error(e)
    
    async def async_chat_completion(self, messages: List[Dict], **kwargs) -> Dict[str, Any]:
        """Async version for multiple requests"""
        try:
            response = await self.async_client.chat.completions.create(
                model=self.model,
                messages=messages,
                **kwargs
            )
            return {
                "content": response.choices[0].message.content,
                "usage": {
                    "prompt_tokens": response.usage.prompt_tokens,
                    "completion_tokens": response.usage.completion_tokens,
                    "total_tokens": response.usage.total_tokens
                },
                "model": response.model
            }
        except Exception as e:
            return self._handle_error(e)
    
    def code_generation(self, prompt: str, language: str = "python") -> Dict[str, Any]:
        """Specialized method for code generation"""
        system_message = f"You are an expert {language} programmer. Write clean, efficient code with proper error handling."
        
        messages = [
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt}
        ]
        
        return self.chat_completion(
            messages, 
            temperature=0.2,
            max_tokens=2000
        )
    
    def _handle_error(self, error: Exception) -> Dict[str, Any]:
        """Comprehensive error handling"""
        error_type = type(error).__name__
        error_mapping = {
            "AuthenticationError": "Invalid API key",
            "RateLimitError": "Rate limit exceeded",
            "APIError": "OpenAI API error",
            "APITimeoutError": "Request timeout"  # FIXED: Correct error name
        }
        
        return {
            "error": True,
            "message": error_mapping.get(error_type, str(error)),
            "type": error_type
        }



In [4]:
# ==================== 2. Google Gemini Setup ====================
class GeminiClient:
    def __init__(self, model: str = "gemini-2.0-flash-exp"):  # Updated model name
        genai.configure(api_key=os.getenv('GEMINI_API_KEY'))
        self.model = genai.GenerativeModel(model)
        self.model_name = model
        
    def generate_content(self, prompt: str, **kwargs) -> Dict[str, Any]:
        """Generate content with safety settings"""
        try:
            generation_config = genai.types.GenerationConfig(
                temperature=kwargs.get('temperature', 0.1),
                max_output_tokens=kwargs.get('max_tokens', 2000),
                top_p=kwargs.get('top_p', 0.95)  # FIXED: Made configurable
            )
            
            safety_settings = [
                {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
                {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
                {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
                {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"}
            ]
            
            response = self.model.generate_content(
                prompt,
                generation_config=generation_config,
                safety_settings=safety_settings
            )
            
            return {
                "content": response.text,
                "safety_ratings": self._parse_safety_ratings(response),
                "model": self.model_name
            }
            
        except Exception as e:
            return self._handle_error(e)
    
    def chat_session(self, history: List[Dict] = None):
        """Start a chat session for back-and-forth programming"""
        try:
            chat = self.model.start_chat(history=history or [])
            return chat
        except Exception as e:
            print(f"Chat session error: {e}")
            return None
    
    def code_analysis(self, code: str, language: str = "python") -> Dict[str, Any]:
        """Analyze code for bugs and improvements"""
        prompt = f"""Analyze this {language} code and provide:
                    1. Potential bugs or issues
                    2. Performance improvements
                    3. Code style suggestions
                    4. Security concerns

                    Code:
                    ```{language}
                    {code}
                    ```
                    """
        return self.generate_content(prompt, temperature=0.1)
    
    def _parse_safety_ratings(self, response) -> List[Dict]:
        """Parse safety ratings from response"""
        safety_ratings = []
        try:
            if hasattr(response, 'candidates') and response.candidates:
                for rating in response.candidates[0].safety_ratings:
                    safety_ratings.append({
                        "category": rating.category.name,
                        "probability": rating.probability.name
                    })
        except Exception:
            pass
        return safety_ratings
    
    def _handle_error(self, error: Exception) -> Dict[str, Any]:
        error_type = type(error).__name__
        return {
            "error": True,
            "message": str(error),
            "type": error_type
        }



In [5]:
# ==================== 3. Anthropic Claude Setup ====================
class ClaudeClient:
    def __init__(self, model: str = "claude-sonnet-4-20250514"):  
        self.client = anthropic.Anthropic(api_key=os.getenv('ANTHROPIC_API_KEY'))
        self.model = model
        self.max_tokens = 4000
        
    def message(self, prompt: str, system: str = None, **kwargs) -> Dict[str, Any]:
        """Send message to Claude with system prompt"""
        try:
            message_params = {
                "model": self.model,
                "max_tokens": kwargs.get('max_tokens', self.max_tokens),
                "temperature": kwargs.get('temperature', 0.1),
                "messages": [{"role": "user", "content": prompt}]
            }
            
            if system:
                message_params["system"] = system
            
            message = self.client.messages.create(**message_params)
            
            return {
                "content": message.content[0].text,
                "usage": {
                    "input_tokens": message.usage.input_tokens,
                    "output_tokens": message.usage.output_tokens
                },
                "model": message.model,
                "stop_reason": message.stop_reason
            }
            
        except Exception as e:
            return self._handle_error(e)
    
    def programming_assistant(self, task: str, code_context: str = None) -> Dict[str, Any]:
        """Specialized method for programming tasks"""
        system_prompt = """You are an expert programming assistant. Follow these rules:
1. Write clean, efficient, and well-documented code
2. Include error handling and edge cases
3. Provide explanations for complex logic
4. Suggest improvements and alternatives"""
        
        if code_context:
            prompt = f"Code context:\n{code_context}\n\nTask: {task}"
        else:
            prompt = task
            
        return self.message(prompt, system=system_prompt, temperature=0.2)
    
    def stream_response(self, prompt: str, system: str = None):
        """Stream response for long-running tasks"""
        try:
            with self.client.messages.stream(
                model=self.model,
                max_tokens=self.max_tokens,
                temperature=0.1,
                system=system,
                messages=[{"role": "user", "content": prompt}]
            ) as stream:
                for text in stream.text_stream:
                    yield text
                    
        except Exception as e:
            yield f"Error: {str(e)}"
    
    def _handle_error(self, error: Exception) -> Dict[str, Any]:
        error_type = type(error).__name__
        error_mapping = {
            "AuthenticationError": "Invalid API key",
            "RateLimitError": "Rate limit exceeded",
            "APIError": "Anthropic API error",
            "OverloadedError": "API overloaded"
        }
        
        return {
            "error": True,
            "message": error_mapping.get(error_type, str(error)),
            "type": error_type
        }



In [6]:
# ==================== 4. DeepSeek Setup ====================
class DeepSeekClient:
    def __init__(self, model: str = "deepseek-chat"):  
        self.client = OpenAI(
            api_key=os.getenv('DEEPSEEK_API_KEY'),
            base_url="https://api.deepseek.com"  
        )
        self.model = model
        
    def chat(self, messages: List[Dict], **kwargs) -> Dict[str, Any]:
        """Chat completion with DeepSeek"""
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                **kwargs
            )
            
            return {
                "content": response.choices[0].message.content,
                "usage": {
                    "prompt_tokens": response.usage.prompt_tokens,
                    "completion_tokens": response.usage.completion_tokens,
                    "total_tokens": response.usage.total_tokens
                },
                "model": response.model
            }
            
        except Exception as e:
            return self._handle_error(e)
    
    def generate_code(self, requirement: str, language: str = "python") -> Dict[str, Any]:
        """Generate code with specific requirements"""
        system_msg = f"You are an expert {language} programmer. Write efficient, production-ready code."
        
        messages = [
            {"role": "system", "content": system_msg},
            {"role": "user", "content": f"Write {language} code for: {requirement}"}
        ]
        
        return self.chat(
            messages,
            temperature=0.1,
            max_tokens=4000
        )
    
    def code_review(self, code: str, language: str = "python") -> Dict[str, Any]:
        """Perform code review"""
        prompt = f"""Review this {language} code and provide:

CRITICAL:
- Security vulnerabilities
- Major bugs
- Performance issues

IMPROVEMENTS:
- Code style
- Best practices
- Documentation

Code:
```{language}
{code}
```
"""
        
        messages = [{"role": "user", "content": prompt}]
        return self.chat(messages, temperature=0.1)
    
    def _handle_error(self, error: Exception) -> Dict[str, Any]:
        return {
            "error": True,
            "message": str(error),
            "type": type(error).__name__
        }



In [7]:
# ==================== 5. Mistral AI Setup ====================
class MistralClient:
    def __init__(self, model: str = "mistral-large-latest"):  
        self.client = Mistral(api_key=os.getenv('MISTRAL_API_KEY'))
        self.model = model
        
    def chat(self, messages: List[Dict], **kwargs) -> Dict[str, Any]:
        """Chat completion with Mistral"""
        try:
            response = self.client.chat.complete(
                model=self.model,
                messages=messages,
                **kwargs
            )
            
            return {
                "content": response.choices[0].message.content,
                "usage": {
                    "prompt_tokens": response.usage.prompt_tokens,
                    "completion_tokens": response.usage.completion_tokens,
                    "total_tokens": response.usage.total_tokens
                },
                "model": response.model
            }
            
        except Exception as e:
            return self._handle_error(e)
    
    async def async_chat(self, messages: List[Dict], **kwargs) -> Dict[str, Any]:
        """Async chat completion"""
        try:
            # Note: Mistral async requires separate async client
            response = await asyncio.to_thread(
                self.client.chat.complete,
                model=self.model,
                messages=messages,
                **kwargs
            )
            
            return {
                "content": response.choices[0].message.content,
                "usage": {
                    "prompt_tokens": response.usage.prompt_tokens,
                    "completion_tokens": response.usage.completion_tokens,
                    "total_tokens": response.usage.total_tokens
                },
                "model": response.model
            }
            
        except Exception as e:
            return self._handle_error(e)
    
    def multi_language_code(self, requirement: str, languages: List[str]) -> Dict[str, Any]:
        """Generate code in multiple languages"""
        prompt = f"""Create implementation for: {requirement}

Provide code in these languages: {', '.join(languages)}
Include proper error handling and documentation for each.
"""
        
        messages = [{"role": "user", "content": prompt}]
        return self.chat(messages, max_tokens=6000)
    
    def _handle_error(self, error: Exception) -> Dict[str, Any]:
        error_type = type(error).__name__
        error_mapping = {
            "APIConnectionError": "Connection error",
            "RateLimitError": "Rate limit exceeded",
            "APIError": "Mistral API error"
        }
        
        return {
            "error": True,
            "message": error_mapping.get(error_type, str(error)),
            "type": error_type
        }



In [8]:
# ==================== 6. Universal Client Manager ====================
class AIModelManager:
    """Universal manager for all AI models"""
    
    def __init__(self):
        self.clients = {
            "openai": OpenAIClient(),
            "gemini": GeminiClient(),
            "claude": ClaudeClient(),
            "deepseek": DeepSeekClient(),
            "mistral": MistralClient()
        }
        
        self.model_preferences = {
            "code_generation": ["deepseek", "claude", "openai"],
            "code_review": ["gemini", "claude", "openai"],
            "quick_tasks": ["gemini", "mistral", "openai"],
            "complex_reasoning": ["claude", "openai", "gemini"]
        }
    
    def get_best_model(self, task_type: str, budget: str = "medium") -> str:
        """Get best model based on task and budget"""
        budget_tiers = {
            "low": ["deepseek", "gemini", "mistral"],
            "medium": ["openai", "claude", "gemini"],
            "high": ["claude", "openai", "mistral"]
        }
        
        available_models = self.model_preferences.get(task_type, ["openai"])
        budget_models = budget_tiers.get(budget, ["openai"])
        
        for model in available_models:
            if model in budget_models:
                return model
        return available_models[0]
    
    def execute_task(self, task: str, task_type: str, **kwargs) -> Dict[str, Any]:
        """Execute task with appropriate model"""
        model_key = self.get_best_model(task_type, kwargs.get('budget', 'medium'))
        client = self.clients.get(model_key)
        
        if not client:
            return {"error": True, "message": f"Client {model_key} not found"}
        
        try:
            if task_type == "code_generation":
                if hasattr(client, 'generate_code'):
                    return client.generate_code(task, kwargs.get('language', 'python'))
                elif hasattr(client, 'code_generation'):
                    return client.code_generation(task, kwargs.get('language', 'python'))
            elif task_type == "code_review":
                if hasattr(client, 'code_analysis'):
                    return client.code_analysis(task, kwargs.get('language', 'python'))
                elif hasattr(client, 'code_review'):
                    return client.code_review(task, kwargs.get('language', 'python'))
            
            # Fallback to generic chat
            messages = [{"role": "user", "content": task}]
            if hasattr(client, 'chat'):
                return client.chat(messages)
            elif hasattr(client, 'chat_completion'):
                return client.chat_completion(messages)
            elif hasattr(client, 'message'):
                return client.message(task)
                
        except Exception as e:
            return {"error": True, "message": str(e)}
    
    async def compare_models(self, prompt: str, models: List[str] = None) -> Dict[str, Any]:
        """Compare responses from multiple models"""
        if not models:
            models = ["openai", "gemini", "claude", "deepseek"]
            
        tasks = []
        for model_name in models:
            client = self.clients.get(model_name)
            if not client:
                tasks.append(asyncio.sleep(0))  # Placeholder
                continue
                
            messages = [{"role": "user", "content": prompt}]
            
            if hasattr(client, 'async_chat_completion'):
                task = client.async_chat_completion(messages)
            elif hasattr(client, 'async_chat'):
                task = client.async_chat(messages)
            else:
                # Fallback to sync wrapped in thread
                if hasattr(client, 'chat'):
                    task = asyncio.to_thread(client.chat, messages)
                elif hasattr(client, 'chat_completion'):
                    task = asyncio.to_thread(client.chat_completion, messages)
                elif hasattr(client, 'message'):
                    task = asyncio.to_thread(client.message, prompt)
                else:
                    task = asyncio.sleep(0)
                    
            tasks.append(task)
            
        results = await asyncio.gather(*tasks, return_exceptions=True)
        
        comparison = {}
        for i, model_name in enumerate(models):
            if isinstance(results[i], Exception):
                comparison[model_name] = {"error": True, "message": str(results[i])}
            else:
                comparison[model_name] = results[i]
                
        return comparison



In [9]:
# ==================== 7. Error Handling & Utilities ====================
class AIErrorHandler:
    """Comprehensive error handling for AI APIs"""
    
    @staticmethod
    def retry_on_error(func, max_retries: int = 3, delay: float = 1.0):
        """Decorator for retrying failed API calls"""
        def wrapper(*args, **kwargs):
            for attempt in range(max_retries):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    if attempt == max_retries - 1:
                        raise e
                    print(f"Attempt {attempt + 1} failed: {e}. Retrying...")
                    time.sleep(delay * (2 ** attempt))  # Exponential backoff
            return None
        return wrapper
    
    @staticmethod
    def validate_response(response: Dict[str, Any]) -> bool:
        """Validate AI response quality"""
        if response.get('error'):
            return False
            
        content = response.get('content', '')
        if not content or len(content.strip()) < 10:
            return False
            
        # Check for common error patterns in AI responses
        error_indicators = [
            "I cannot", "I'm unable", "as an AI", "I don't have",
            "I'm not able", "I cannot provide", "I'm sorry"
        ]
        
        content_lower = content.lower()
        return not any(indicator in content_lower for indicator in error_indicators)


class CodeQualityChecker:
    """Utilities for checking generated code quality"""
    
    @staticmethod
    def check_python_syntax(code: str) -> bool:
        """Check if Python code is syntactically valid"""
        try:
            compile(code, '<string>', 'exec')
            return True
        except SyntaxError:
            return False
    
    @staticmethod
    def extract_code_blocks(text: str, language: str = "python") -> List[str]:
        """Extract code blocks from AI responses"""
        import re
        pattern = f"```{language}\\s*\\n(.*?)```"
        matches = re.findall(pattern, text, re.DOTALL | re.IGNORECASE)
        if not matches:
            # Try without language specifier
            pattern = r"```(.*?)```"
            matches = re.findall(pattern, text, re.DOTALL)
        return [match.strip() for match in matches]


In [11]:
# ==================== USAGE EXAMPLES ====================
if __name__ == "__main__":
    # Example 1: OpenAI
    print("=== OpenAI Example ===")
    openai_client = OpenAIClient()
    result = openai_client.code_generation(
        "Create a Python function to calculate fibonacci sequence with memoization"
    )
    print(result.get("content", result))

=== OpenAI Example ===
{'error': True, 'message': 'Rate limit exceeded', 'type': 'RateLimitError'}


In [12]:
# ==================== USAGE EXAMPLES ====================
if __name__ == "__main__":
    # Example 2: Gemini
    print("\n=== Gemini Example ===")
    gemini_client = GeminiClient()
    analysis = gemini_client.code_analysis("""
def calculate_average(numbers):
    return sum(numbers) / len(numbers)
""")
    print(analysis.get("content", analysis))


=== Gemini Example ===
{'error': True, 'message': '429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash-exp\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash-exp\nPlease retry in 54.767817026s. [links {\n  description: "Learn more about Gemini API quotas"\n  url: "https://ai.google.dev/gemini-api/docs/rate-limits"\n}\n, violations {\n  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"\n  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"\n  quota_dimensions {\n    key: "model"\n    value: "gemini-2.0-flash-exp"\n

In [13]:
# ==================== USAGE EXAMPLES ====================
if __name__ == "__main__":
    # Example 3: Claude
    print("\n=== Claude Example ===")
    claude_client = ClaudeClient()
    result = claude_client.programming_assistant(
        "Create a Python class for handling API requests with retry logic"
    )
    print(result.get("content", result))


=== Claude Example ===
{'error': True, 'message': "Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits.'}, 'request_id': 'req_011CViabYH3vCKXbGnfyvZFs'}", 'type': 'BadRequestError'}


In [14]:
# ==================== USAGE EXAMPLES ====================
if __name__ == "__main__":
    # Example 4: DeepSeek
    print("\n=== DeepSeek Example ===")
    deepseek_client = DeepSeekClient()
    code_result = deepseek_client.generate_code(
        "Create a FastAPI endpoint for user authentication with JWT tokens"
    )
    print(code_result.get("content", code_result))


=== DeepSeek Example ===
{'error': True, 'message': "Error code: 402 - {'error': {'message': 'Insufficient Balance', 'type': 'unknown_error', 'param': None, 'code': 'invalid_request_error'}}", 'type': 'APIStatusError'}


In [15]:
# ==================== USAGE EXAMPLES ====================
if __name__ == "__main__":
    # Example 5: Model Manager
    print("\n=== Model Manager Example ===")
    manager = AIModelManager()
    result = manager.execute_task(
        "Create a Python data validation class using Pydantic",
        "code_generation",
        budget="medium"
    )
    print(result.get("content", result))



=== Model Manager Example ===
{'error': True, 'message': "Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits.'}, 'request_id': 'req_011CViargBDyPjoUCFs716By'}", 'type': 'BadRequestError'}


In [10]:
# ==================== USAGE EXAMPLES ====================
if __name__ == "__main__":
    # Example 1: OpenAI
    print("=== OpenAI Example ===")
    openai_client = OpenAIClient()
    result = openai_client.code_generation(
        "Create a Python function to calculate fibonacci sequence with memoization"
    )
    print(result.get("content", result))
    
    # Example 2: Gemini
    print("\n=== Gemini Example ===")
    gemini_client = GeminiClient()
    analysis = gemini_client.code_analysis("""
def calculate_average(numbers):
    return sum(numbers) / len(numbers)
""")
    print(analysis.get("content", analysis))
    
    # Example 3: Claude
    print("\n=== Claude Example ===")
    claude_client = ClaudeClient()
    result = claude_client.programming_assistant(
        "Create a Python class for handling API requests with retry logic"
    )
    print(result.get("content", result))
    
    # Example 4: DeepSeek
    print("\n=== DeepSeek Example ===")
    deepseek_client = DeepSeekClient()
    code_result = deepseek_client.generate_code(
        "Create a FastAPI endpoint for user authentication with JWT tokens"
    )
    print(code_result.get("content", code_result))
    
    # Example 5: Model Manager
    print("\n=== Model Manager Example ===")
    manager = AIModelManager()
    result = manager.execute_task(
        "Create a Python data validation class using Pydantic",
        "code_generation",
        budget="medium"
    )
    print(result.get("content", result))


=== OpenAI Example ===
{'error': True, 'message': 'Rate limit exceeded', 'type': 'RateLimitError'}

=== Gemini Example ===
{'error': True, 'message': '429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash-exp\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash-exp\nPlease retry in 39.78038924s. [links {\n  description: "Learn more about Gemini API quotas"\n  url: "https://ai.google.dev/gemini-api/docs/rate-limits"\n}\n, violations {\n  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"\n  quota_id: "GenerateRequestsPerMinutePerProj